# Dependencies
First, we check that the notebook is setup for rendering the simulations!

In [1]:
try:
    import webgui_jupyter_widgets
    from packaging.version import parse
    assert parse(webgui_jupyter_widgets.__version__) >= parse("0.2.18")
    print('Everything good!')
except:
    print("\x1b[31mYou need to update webgui_jupyter_widgets by running: \x1b[0m\npython3 -m pip install --upgrade webgui_jupyter_widgets")

Everything good!


Now we import the ngsolve library that we will use for the simulations.

In [2]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *   # Opencascade for geometry modeling

# Setting up the Simulation

The first task is to setup a mesh that we can simulate our equations on.

## The geometry
We're going to look at conduction inside a circle (imagine a slice from a steel bar), with convection on the outside. So lets create a circular mesh.

In [17]:
# Make a radius 1 circle
geom = Circle((0,0), 1).Face()
# Turn it into a 2D mesh, with triangles with a maximum edge length of 0.1
mesh = Mesh(OCCGeometry(geom, dim=2).GenerateMesh(maxh=0.1))
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## The equations

We now need to generate a finite element space where we're going to solve our
equations on. This is different to the mesh, as the points of the mesh might be
mapped onto the equations in many different ways.

In [20]:
fes = H1(mesh, order=2, dirichlet="default")
fes.ndof

1442

The equations themselves are the heat diffusion equation
$$
\rho\,C_p\frac{\partial T}{\partial t} = -\rho\,C_p\,\bm{v}
  \cdot\nabla\,T - \nabla\cdot\bm{q} - \bm{\tau}:\nabla\,\bm{v} -
  p\,\nabla\cdot\bm{v} +\sigma_{energy}
$$

which for stationary solids $\bm{v}=0$, becomes

$$
\rho\,C_p\frac{\partial T}{\partial t} = - \nabla\cdot\bm{q} +\sigma_{energy}
$$

Inserting fourier's law,
$$
\rho\,C_p\frac{\partial T}{\partial t} = - k\nabla\cdot\nabla\bm{T} +\sigma_{energy}
$$

To make this a finite element problem, we define a trial function $v$ we "project" this equation onto

$$
\int_{\Omega} \rho\,C_p\frac{\partial T}{\partial t} v \,{\rm d}\bm{r} + \int_{\Omega} (k\nabla\cdot\nabla\bm{T})\,v\,{\rm d}\bm{r} =  \int_{\Omega}\sigma_{energy}\,v\,{\rm d}\bm{r}
$$

where this is an integration over the whole domain, $\Omega$. We can then use the integration by parts identity

$$
[u'\, v]_{\Omega} = \int_{\Omega} u'\,v'\,{\rm d}\bm{r} +  \int_{\Omega} u''\,v\,{\rm d}\bm{r} 
$$